In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from simpledbf import Dbf5
import pyxlsb
import os
import time
import random
from shapely.geometry import MultiLineString, LineString, Point
import math
import shapely

import create_gtfs_from_basicinfo
import sys, os
import zipfile
import shutil

/Users/frankiemacbook/opt/miniconda3/envs/py3/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [2]:
start_time = time.time()

rootdir = '/Users/frankiemacbook/OneDrive - VicGov/VITM/GTFS/Ref case'

fullpaths = map(lambda name: os.path.join(rootdir, name), os.listdir(rootdir))
dirs = [x for x in fullpaths if os.path.isdir(x)]
years = sorted([x.rsplit('/', 1)[-1] for x in dirs])
years

['2018', '2026', '2031', '2036', '2041', '2051']

In [3]:
data = pd.DataFrame()
for year in years:
    dbf = Dbf5(rootdir+'/'+year+'/PTLINK_ALL_Y'+year+'_VR19_Ref_C.DBF')
    temp = dbf.to_dataframe()
    temp.insert(0,'year',year)
    data = data.append(temp, ignore_index=True)
data = data[['PERIOD','A','B','VEHNAME','LINENO','LINKSEQ','STOPA','STOPB','NAME','LONGNAME','year']]
data['YEAR_LINE_PERIOD'] = data['year'] + "_" + data['LINENO'].apply(str) + "_" + data['PERIOD']
data['YEAR_LINE'] = data['year'] + "_" + data['LINENO'].apply(str)
data['A_B'] = data['A'].apply(str) + "_" + data['B'].apply(str)

In [4]:
tram_names = data.loc[data['VEHNAME'].str.contains("^(?:Tram)")]['VEHNAME'].unique().tolist()
vline_names = data.loc[data['VEHNAME'].str.contains("^(?:V[Ll]ine)")]['VEHNAME'].unique().tolist() + ['SP2']
train_names = data.loc[(data['VEHNAME'].str.contains("^(?:Metro)")) | \
                       (data['VEHNAME'].str.contains("^(?:HCMT)")) | \
                       (data['VEHNAME'].str.contains("^(?:COMENG)")) | \
                       (data['VEHNAME'].str.contains("^(?:SRL)"))]['VEHNAME'].unique().tolist()

replace_dict = {'Bus':'bus','SkyBus':'skybus'}
for name in tram_names:
    replace_dict[name] = 'tram'
for name in vline_names:
    replace_dict[name] = 'vline'
for name in train_names:
    replace_dict[name] = 'train'

data['VEHNAME'] = data['VEHNAME'].replace(replace_dict)
data = data.rename(columns = {'VEHNAME':'mode'})
all_modes = sorted(data['mode'].unique().tolist())
all_modes

['bus', 'skybus', 'train', 'tram', 'vline']

In [5]:
ignore_fields_list = ['REGION','LINKC_IP','LINKC_PM','LINKC_OP', 'LANES_AM', 'LANES_IP', 'LANES_PM', 'LANES_OP', 'PSPD_AM', 'PSPD_IP', 'PSPD_PM', 'PSPD_OP', 'TOLLROAD', \
                      'TOLLGANT', 'TOLLENTRY', 'TOLLEXIT', 'TURN_BAN', 'CLEARWAY', 'ROADTYPE', 'RD_NAME', 'LASTPROJ', 'LX', 'LX_CODE', 'CROSSING', 'LXCLOSE', 'CTIME', \
                      'RAIL_SPD', 'TRAM_SPD3', 'BFLG_AM', 'BFLG_PM', 'SL', 'SL_CODE', 'TTR', 'TRN_CRDN', 'VLN_CRDN', 'TRM_CRDN', 'CRDN_DIR', 'FFSPD_AM', \
                      'FFSPD_IP', 'FFSPD_PM', 'FFSPD_OP', 'FFTIME_AM', 'FFTIME_IP', 'FFTIME_PM', 'FFTIME_OP', 'CSPD_AM', 'CSPD_IP', 'CSPD_PM', 'CSPD_OP', 'TIME_AM', \
                      'TIME_IP', 'TIME_PM', 'TIME_OP', 'TRSTIME_AM', 'TRSTIME_IP', 'TRSTIME_PM', 'TRSTIME_OP', 'TRMTIME_AM', 'TRMTIME_IP', 'TRMTIME_PM', 'TRMTIME_OP', \
                      'BUSTIME_AM', 'BUSTIME_IP', 'BUSTIME_PM', 'BUSTIME_OP', 'SMBTIME_AM', 'SMBTIME_IP', 'SMBTIME_PM', 'SMBTIME_OP', 'SKBTIME_AM', 'SKBTIME_IP', \
                      'SKBTIME_PM', 'SKBTIME_OP', 'SPBTIME_AM', 'SPBTIME_IP', 'SPBTIME_PM', 'SPBTIME_OP', 'VEH_AM', 'VEH_IP', 'VEH_PM', 'VEH_OP', 'VEH_WD', 'PVV_AM', \
                      'PVV_IP', 'PVV_PM', 'PVV_OP', 'PVV_WD', 'AP_EMP_AM', 'AP_EMP_IP', 'AP_EMP_PM', 'AP_EMP_OP', 'AP_EMP_WD', 'AP_PAS_AM', 'AP_PAS_IP', 'AP_PAS_PM', \
                      'AP_PAS_OP', 'AP_PAS_WD', 'TRUCK_AM', 'TRUCK_IP', 'TRUCK_PM', 'TRUCK_OP', 'TRUCK_WD', 'RIGID_AM', 'RIGID_IP', 'RIGID_PM', 'RIGID_OP', 'RIGID_WD', \
                      'ARTIC_AM', 'ARTIC_IP', 'ARTIC_PM', 'ARTIC_OP', 'ARTIC_WD', 'BDBLE_AM', 'BDBLE_IP', 'BDBLE_PM', 'BDBLE_OP', 'BDBLE_WD', 'HPFV_AM', 'HPFV_IP', \
                      'HPFV_PM', 'HPFV_OP', 'HPFV_WD', 'PCU_AM', 'PCU_IP', 'PCU_PM', 'PCU_OP', 'PCU_WD', 'VEH_VKT_AM', 'VEH_VKT_IP', 'VEH_VKT_PM', 'VEH_VKT_OP', \
                      'PVV_VKT_AM', 'PVV_VKT_IP', 'PVV_VKT_PM', 'PVV_VKT_OP', 'HCV_VKT_AM', 'HCV_VKT_IP', 'HCV_VKT_PM', 'HCV_VKT_OP', 'VEH_VHT_AM', 'VEH_VHT_IP', \
                      'VEH_VHT_PM', 'VEH_VHT_OP', 'PVV_VHT_AM', 'PVV_VHT_IP', 'PVV_VHT_PM', 'PVV_VHT_OP', 'HCV_VHT_AM', 'HCV_VHT_IP', 'HCV_VHT_PM', 'HCV_VHT_OP', \
                      'HYCAP_AM', 'HYCAP_IP', 'HYCAP_PM', 'HYCAP_OP', 'VC_AM', 'VC_IP', 'VC_PM', 'VC_OP', 'PT_AM', 'PT_IP', 'PT_PM', 'PT_OP', 'PT_WD', 'TRAIN_AM', \
                      'TRAIN_IP', 'TRAIN_PM', 'TRAIN_OP', 'TRAIN_WD', 'TRAM_AM', 'TRAM_IP', 'TRAM_PM', 'TRAM_OP', 'TRAM_WD', 'BUS_AM', 'BUS_IP', 'BUS_PM', 'BUS_OP', \
                      'BUS_WD', 'VLINE_AM', 'VLINE_IP', 'VLINE_PM', 'VLINE_OP', 'VLINE_WD', 'WLK_AE_AM', 'WLK_AE_IP', 'WLK_AE_PM', 'WLK_AE_OP', 'WLK_AE_WD', \
                      'PNR_AE_AM', 'PNR_AE_IP', 'PNR_AE_PM', 'PNR_AE_OP', 'PNR_AE_WD', 'PNR_VH_AM', 'PNR_VH_IP', 'PNR_VH_PM', 'PNR_VH_OP', 'PNR_VH_WD', 'TFTMBU_AM', \
                      'TFTMBU_IP', 'TFTMBU_PM', 'TFTMBU_OP', 'TFTMBU_WD', 'TFRLRL_AM', 'TFRLRL_IP', 'TFRLRL_PM', 'TFRLRL_OP', 'TFRLRL_WD', 'TRNNVEH_AM', 'TRNNVEH_IP', \
                      'TRNNVEH_PM', 'TRNNVEH_OP', 'TRMNVEH_AM', 'TRMNVEH_IP', 'TRMNVEH_PM', 'TRMNVEH_OP', 'BUSNVEH_AM', 'BUSNVEH_IP', 'BUSNVEH_PM', 'BUSNVEH_OP', \
                      'VLNNVEH_AM', 'VLNNVEH_IP', 'VLNNVEH_PM', 'VLNNVEH_OP', 'TRNCAP_AM', 'TRNCAP_IP', 'TRNCAP_PM', 'TRNCAP_OP', 'TRN_VC_AM', 'TRN_VC_IP', \
                      'TRN_VC_PM', 'TRN_VC_OP', 'TRMCAP_AM', 'TRMCAP_IP', 'TRMCAP_PM', 'TRMCAP_OP', 'TRM_VC_AM', 'TRM_VC_IP', 'TRM_VC_PM', 'TRM_VC_OP', 'BUSCAP_AM', \
                      'BUSCAP_IP', 'BUSCAP_PM', 'BUSCAP_OP', 'BUS_VC_AM', 'BUS_VC_IP', 'BUS_VC_PM', 'BUS_VC_OP', 'VLNCAP_AM', 'VLNCAP_IP', 'VLNCAP_PM', 'VLNCAP_OP', \
                      'VLN_VC_AM', 'VLN_VC_IP', 'VLN_VC_PM', 'VLN_VC_OP', 'PTCAP_AM', 'PTCAP_IP', 'PTCAP_PM', 'PTCAP_OP', 'PT_VC_AM', 'PT_VC_IP', 'PT_VC_PM', 'PT_VC_OP']

network = pd.DataFrame()
for year in years[:1]:
    temp = gpd.read_file(rootdir+'/'+year+'/SUMMARY_LOADED_NETWORK_LINKS_Y'+year+'_VR19_Ref_C.shp',ignore_fields=ignore_fields_list)
    temp = temp.loc[~temp['LINKC_AM'].isin([1,-1])]
    temp['A_B'] = temp['A'].apply(str) + "_" + temp['B'].apply(str)
    temp = temp.set_index('A_B')
    missing_index = temp.index.difference(network.index)
    network = network.append(temp.loc[missing_index, :])
network = network.reset_index().rename(columns = {'index':'A_B'})
network = gpd.GeoDataFrame(network, geometry='geometry')
network.crs=('epsg:20255')
network['DISTANCE'] = network['geometry'].length

In [6]:
pt_reporting = pd.DataFrame()
for year in years:
    temp = pd.read_excel(rootdir+'/'+year+'/DetailedPTReporting_v200619_Y'+year+'_VR19_Ref_C.xlsb', sheet_name='Line Summary', engine='pyxlsb', skiprows=2)
    temp.insert(0,'year',year)
    pt_reporting = pt_reporting.append(temp, ignore_index=True)

In [79]:
temp_data = data.merge(network[['DISTANCE','A_B','geometry']], how='left', on='A_B').reset_index()

In [29]:
segs = temp_data.groupby('A_B').first()[['index','mode','A','B','STOPA','STOPB','DISTANCE','geometry']]

In [30]:
missing_links = segs.loc[segs['DISTANCE'].isnull()]

In [31]:
def find_shortest_path(row):
    
    if row['node_path'] is None:
        
        global rows_processed, segs_not_found
        start_time = time.time()
        too_long = False
        bounding_nodes = [temp_data.loc[row['index']-1,'A'],temp_data.loc[row['index']+1,'B']]
        mode = row['mode']
        start_node = row['A']
        end_node = row['B']
        depth = 1
        max_depth = 500
        max_time = 90
        node_tree = [[start_node]]
        rows_processed +=1
        print('Finding missing nodes: %.1f' % (100 * rows_processed / total_rows) + '% pairs processed                      ', end='\r')
        if row['next_nodes'] is not None:
            node_tree = [node_tree[0] + row['next_nodes']]
            max_depth = 500
            max_time = 120
        while True:
            if (time.time() - start_time > 30) and too_long == False:
                print('Node pair ('+str(start_node)+', '+str(end_node)+') is taking a long time to process', end='\r')
                too_long = True
            new_tree = []
            for nodes in node_tree:
                try:
                    if mode in ['train','vline']:
                        new_nodes = network.loc[(network['A'] == nodes[-1]) & (network['LINKC_AM'] == 42), 'B'].to_list()
                    else:
                        new_nodes = network.loc[(network['A'] == nodes[-1]) & (network['LINKC_AM'] != 42), 'B'].to_list()
                    if any(x == end_node for x in new_nodes):
                        nodes.append(end_node)
                        row['node_path'] = split_list(nodes)
                        return row
                    if (depth >= 5) and (len(new_nodes) > 2) and (random.random() < 0.5):
                        new_nodes = []
                    for new_node in new_nodes:
                        if new_node not in nodes + bounding_nodes:
                            nodes_copy = nodes.copy()
                            nodes_copy.append(new_node)
                            new_tree.append(nodes_copy)
                except:
                    None
            node_tree = new_tree
            depth += 1
            if (depth == max_depth) or (len(node_tree) > 50000) or (time.time() - start_time > max_time):
                segs_not_found.append('"A"='+str(start_node)+' OR "B"='+str(end_node))
#                 print('Node pair ('+str(start_node)+', '+str(end_node)+') not found at depth='+str(depth)+' and tree length='+str(len(node_tree)), end='\r')
                row['node_path'] = None
    return row

def split_list(l):
    new_list = []
    for i in range(len(l)-1):
        new_list.append((l[i],l[i+1]))
    return new_list

missing_links.insert(0,'node_path',None)
missing_links.insert(0,'next_nodes',None)

next_nodes = {'23237_23051':[23212,23191,23188],
              '14610_23262':[23377,23375,23376],
              '10322_19292':[30688,30682,30677],
              '220608_44422':[220609,220610,220612],
              '23051_23237':[23059,23060,23066],
              '23262_14610':[23257,23253,23252],
              '44422_220608':[220102,220103,220104],
              '19292_10322':[30935,30937,30909],
              '27950_26914':[27933,27928,27924],
              '18386_233825':[279141,279140,279139],
              '233825_18386':[279144,279145,279146],
              '223580_15575':[223581,224430,224431]}

for k in next_nodes:
    if k in missing_links.index:
        missing_links.at[k,'next_nodes'] = next_nodes[k]

total_rows = missing_links.shape[0]
rows_processed = 0

segs_not_found = []
missing_links = missing_links.apply(find_shortest_path, axis=1)
if len(segs_not_found) > 0:
    print("\nSegments not found:")
    for s in segs_not_found:
        print(s)
        
# import os
# os.system("printf '\a'")

In [80]:
def pad_stops(group):
    group['STOPB'].iloc[:-1] = 0
    group['STOPA'].iloc[1:] = 0
    return group

if missing_links.size != 0:
    
    ml_expand = missing_links[['index','A','B','STOPA','STOPB']] \
        .merge(missing_links.node_path.explode(), right_index = True, left_index = True) \
        .dropna() \
        .sort_values(['index','A','B']) \
        .reset_index(drop = True)

    ml_expand['A_B'] = ml_expand['A'].apply(str) + "_" + ml_expand['B'].apply(str)

    ml_expand = ml_expand.sort_values(['A_B','index'])
    ml_expand = ml_expand.groupby('A_B').apply(pad_stops)

    temp_data = temp_data.merge(ml_expand[['A_B','STOPA','STOPB','node_path']], how='left', on='A_B', suffixes=('','_y')).sort_values(['year','LINENO','PERIOD','LINKSEQ']).reset_index(drop=True)

    temp_data[['new_A','new_B']] = pd.DataFrame(temp_data.loc[temp_data['node_path'].notnull(), 'node_path'].tolist(), index=temp_data.loc[temp_data['node_path'].notnull()].index)

    temp_data['STOPA'] = np.where(temp_data['node_path'].notnull(), temp_data['STOPA_y'], temp_data['STOPA'])
    temp_data['STOPB'] = np.where(temp_data['node_path'].notnull(), temp_data['STOPB_y'], temp_data['STOPB'])
    temp_data['A'] = np.where(temp_data['node_path'].notnull(), temp_data['new_A'], temp_data['A'])
    temp_data['B'] = np.where(temp_data['node_path'].notnull(), temp_data['new_B'], temp_data['B'])

    temp_data = temp_data.drop(['node_path', 'new_A', 'new_B', 'STOPA_y', 'STOPB_y'], axis=1)

    temp_data['A'] = temp_data['A'].apply(int)
    temp_data['B'] = temp_data['B'].apply(int)

    temp_data['A_B'] = temp_data['A'].apply(str) + "_" + temp_data['B'].apply(str)

df = temp_data.drop(['index','DISTANCE','geometry'], axis=1).merge(network[['LINKC_AM','A_B','DISTANCE','BUS_SPD','geometry']], how='left', on='A_B').replace(np.nan,"NaN")

In [174]:
# short_links = network.loc[network['DISTANCE'] < 5]
# for index, row in short_links.iterrows():
#     next_nodes = network.loc[(network['A'] == row['B']) & (network['B'] != row['A']) & (network['LINKC_AM'] == row['LINKC_AM'])]
#     if next_nodes.shape[0] == 1:
#         next_nodes = next_nodes.iloc[0]
#         if network.loc[(network['B'] == row['B']) & (~network['A'].isin([row['A'],next_nodes['B']]))].shape[0] == 0:
#             network.loc[index,'B'] = next_nodes['B']
#             network.loc[index,'geometry'] = LineString(list(network.loc[index,'geometry'].coords) + list(next_nodes['geometry'].coords[1:]))
#             network.loc[index,'DISTANCE'] = network.loc[index,'DISTANCE'] + next_nodes['DISTANCE']
#             network.loc[index,'A_B'] = str(network.loc[index,'A']) + '_' + str(network.loc[index,'B'])
#     else:
#         prev_nodes = network.loc[(network['B'] == row['A']) & (network['A'] != row['B']) & (network['LINKC_AM'] == row['LINKC_AM'])]
#         if prev_nodes.shape[0] == 1:
#             prev_nodes = prev_nodes.iloc[0]
#             if network.loc[(network['A'] == row['A']) & (~network['B'].isin([row['B'],prev_nodes['A']]))].shape[0] == 0:
#                 network.loc[index,'A'] = prev_nodes['A']
#                 network.loc[index,'geometry'] = LineString(list(prev_nodes['geometry'].coords[1:]) + list(network.loc[index,'geometry'].coords))
#                 network.loc[index,'DISTANCE'] = network.loc[index,'DISTANCE'] + prev_nodes['DISTANCE']
#                 network.loc[index,'A_B'] = str(network.loc[index,'A']) + '_' + str(network.loc[index,'B'])

In [11]:
# for k in next_nodes:
#     if k in missing_links.index:
#         missing_links.at[k,'next_nodes'] = next_nodes[k]

# total_rows = missing_links.loc[missing_links['node_path'] == 'NaN'].shape[0]
# rows_processed = 0

# segs_not_found = []
# missing_links = missing_links.apply(find_shortest_path, axis=1)
# if len(segs_not_found) > 0:
#     print("\nSegments not found:")
#     for s in segs_not_found:
#         print(s)

In [13]:
# del network

# del temp_data

# del temp
# del missing_index
# del segs
# del missing_links
# del ml_expand

In [81]:
df['LINKSEQ'] = df.groupby('YEAR_LINE_PERIOD').cumcount()
df = df.merge(df.groupby('YEAR_LINE_PERIOD')['B'].apply(list).apply(lambda x: '_'.join([str(y) for y in x])).reset_index().rename(columns={'B':'ROUTE_LIST'}), how='left', on='YEAR_LINE_PERIOD')

unique_routes = df.groupby('ROUTE_LIST').first().reset_index().reset_index().rename(columns={'index':'UNIQUE_ROUTE'})[['ROUTE_LIST','UNIQUE_ROUTE','YEAR_LINE_PERIOD']]

df = df.merge(unique_routes.drop('YEAR_LINE_PERIOD',axis=1), how='left', on='ROUTE_LIST').drop('ROUTE_LIST',axis=1)
df['UNIQUE_ROUTE_SEQ'] = df['UNIQUE_ROUTE'].apply(str) + '_' + df['LINKSEQ'].apply(str)

def assert_route_termini(group):
    group['STOPA'].iloc[0] = 1
    group['STOPB'].iloc[-1] = 1
    return group

df_unique = df.loc[df['YEAR_LINE_PERIOD'].isin(unique_routes['YEAR_LINE_PERIOD'].to_list())].sort_values(['UNIQUE_ROUTE','LINKSEQ']).groupby('UNIQUE_ROUTE').apply(assert_route_termini).sort_values(['UNIQUE_ROUTE','LINKSEQ']).reset_index(drop=True)
df_unique = df_unique[['A_B','A','B','STOPA','STOPB','LINKC_AM','BUS_SPD','DISTANCE','UNIQUE_ROUTE','UNIQUE_ROUTE_SEQ','LINKSEQ','mode','geometry']]

unique_AB_to_split = df_unique.groupby('A_B').first().reset_index()

unique_AB_to_split = unique_AB_to_split.loc[(unique_AB_to_split['mode'].isin(['bus','tram'])) & (~unique_AB_to_split['LINKC_AM'].isin([18,20,25])) & (unique_AB_to_split['BUS_SPD'] != 80)][['A_B','A','B','LINKC_AM','DISTANCE','geometry']]

In [91]:
unique_AB_to_split = gpd.GeoDataFrame(unique_AB_to_split,geometry='geometry')

In [99]:
unique_nodes = unique_AB_to_split.drop_duplicates(subset=['A'])[['A','geometry']].rename(columns={'A':'node'})
unique_nodes.geometry = unique_nodes.geometry.apply(lambda line: Point(line.coords[0]))
temp_B_nodes = unique_AB_to_split.drop_duplicates(subset=['B'])[['B','geometry']].rename(columns={'B':'node'})
temp_B_nodes.geometry = temp_B_nodes.geometry.apply(lambda line: Point(line.coords[-1]))
unique_nodes = unique_nodes.append(temp_B_nodes.loc[~temp_B_nodes['node'].isin(unique_nodes['node'].to_list())])
unique_nodes = gpd.GeoDataFrame(unique_nodes, geometry='geometry')

print(unique_nodes.shape[0])

25815


In [101]:
total_rows = unique_AB_to_split.shape[0]
rows_processed = 0

def split_line(line, distance, segments):
    global unique_nodes, geoms
    line_segs = [line]
    for i in range(segments - 1):
        line = line_segs[-1]
        line_segs = line_segs[:-1]
        coords = list(line.coords)
        for i, p in enumerate(coords):
            dist = line.project(Point(p))
            if dist == distance:
                dist_to_nearest = unique_nodes.distance(Point(coords[i])).nsmallest(1)
                if dist_to_nearest.iloc[0] == 0:
                    line_segs += [LineString(coords[:i+1]), LineString(coords[i:])]
#                     print('existing node: '+','.join(str(c) for c in coords[i]))
                else:
                    if dist_to_nearest.iloc[0] < 5:
                        nearest_point = unique_nodes.loc[dist_to_nearest.index,'geometry'].iloc[0]
                        line_segs += [LineString(coords[:i] + [(nearest_point.x,nearest_point.y)]), LineString([(nearest_point.x,nearest_point.y)] + coords[i+1:])]
#                         print('nearby node: '+','.join(str(c) for c in coords[i])+' -> '+','.join(str(c) for c in [nearest_point.x,nearest_point.y]))
                    else:
                        line_segs += [LineString(coords[:i+1]), LineString(coords[i:])]
                        unique_nodes = unique_nodes.append(gpd.GeoDataFrame(pd.DataFrame([[np.nan,Point(coords[i])]],columns=['node','geometry']),geometry='geometry'),ignore_index=True)
#                         print('new node: '+','.join(str(c) for c in coords[i]))
                break
            if dist > distance:
                cp = line.interpolate(distance)
                dist_to_nearest = unique_nodes.distance(cp).nsmallest(1)
                if dist_to_nearest.iloc[0] == 0:
                    line_segs += [
                        LineString(coords[:i] + [(cp.x, cp.y)]),
                        LineString([(cp.x, cp.y)] + coords[i:])]
#                     print('existing node: '+','.join(str(c) for c in [cp.x, cp.y]))
                else:
                    dist_to_nearest = unique_nodes.distance(cp).nsmallest(1)
                    if dist_to_nearest.iloc[0] < 5:
                        nearest_point = unique_nodes.loc[dist_to_nearest.index,'geometry'].iloc[0]
                        line_segs += [LineString(coords[:i] + [(nearest_point.x,nearest_point.y)]), LineString([(nearest_point.x,nearest_point.y)] + coords[i:])]
#                         print('nearby node: '+','.join(str(c) for c in [cp.x, cp.y])+' -> '+','.join(str(c) for c in [nearest_point.x,nearest_point.y]))
                    else:
                        line_segs += [
                            LineString(coords[:i] + [(cp.x, cp.y)]),
                            LineString([(cp.x, cp.y)] + coords[i:])]
                        unique_nodes = unique_nodes.append(gpd.GeoDataFrame(pd.DataFrame([[np.nan,cp]],columns=['node','geometry']),geometry='geometry'),ignore_index=True)
#                         print('new node: '+','.join(str(c) for c in [cp.x, cp.y]))
                break
    return line_segs
        
def add_stops(row):
    global rows_processed
    rows_processed +=1
    print('Splitting long segments: %.1f' % (100 * rows_processed / total_rows) + '% processed', end='\r')
    line = row['geometry']
    dist = row['DISTANCE']
    if dist > 250:
        segs = math.ceil(dist / 200)
        seg_length = dist / segs
        return split_line(line, seg_length, segs)
    return None

unique_AB_to_split['new_segments'] = unique_AB_to_split.apply(add_stops, axis=1)

print('Splitting long segments: finished           ')
print(unique_nodes.shape[0])

Splitting long segments: finished           
46503


In [101]:
total_rows = unique_AB_to_split.shape[0]
rows_processed = 0

def split_line(line, distance, segments):
    line_segs = [line]
    for i in range(segments - 1):
        line = line_segs[-1]
        line_segs = line_segs[:-1]
        coords = list(line.coords)
        for i, p in enumerate(coords):
            pd = line.project(Point(p))
            if pd == distance:
                line_segs += [LineString(coords[:i+1]), LineString(coords[i:])]
            if pd > distance:
                cp = line.interpolate(distance, )
                line_segs += [
                    LineString(coords[:i] + [(cp.x, cp.y)]),
                    LineString([(cp.x, cp.y)] + coords[i:])]
    return line_segs
        
def add_stops(row):
    global rows_processed
    rows_processed +=1
    print('Splitting long segments: %.1f' % (100 * rows_processed / total_rows) + '% processed', end='\r')
    line = row['geometry']
    dist = line.length
    if dist > 150:
        segs = math.ceil(dist / 150)
        seg_length = dist / segs
        return split_line(line, seg_length, segs)
    return None

unique_AB_to_split['new_segments'] = unique_AB_to_split.apply(add_stops, axis=1)

print('Splitting long segments: finished           ')

Splitting long segments: finished           
46503


In [103]:
unique_AB_to_split = unique_AB_to_split.loc[unique_AB_to_split['new_segments'].notnull()]

unique_AB_to_split['STOPA'] = 1
unique_AB_to_split['STOPB'] = 1

unique_AB_to_split = unique_AB_to_split[['A','B','A_B','STOPA','STOPB','geometry']] \
    .merge(unique_AB_to_split['new_segments'].explode(), right_index = True, left_index = True) \
    .dropna() \
    .reset_index(drop = True)

unique_AB_to_split = unique_AB_to_split.drop(['geometry'], axis=1).rename(columns={'new_segments':'geometry'})

/Users/frankiemacbook/opt/miniconda3/envs/py3/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [144]:
# unique_points = df_unique.drop_duplicates(subset=['A'])[['A','geometry']].rename(columns={'A':'node'})

# temp_B_points = df_unique.drop_duplicates(subset=['B'])[['B','geometry']].rename(columns={'B':'node'})

# unique_points = unique_points.append(temp_B_points.loc[~temp_B_points['node'].isin(unique_points['node'].to_list())])
# unique_points.geometry = unique_points.geometry.apply(lambda line: Point(line.coords[0]))
# unique_points = gpd.GeoDataFrame(unique_points, geometry='geometry')

In [105]:
max_node = df[['A','B']].max().max()

def insert_nodes(group):
    global max_node
    if group.shape[0] > 1:
        new_nodes = list(range(max_node+1,max_node+group.shape[0]))
        group['B'].iloc[:-1] = new_nodes
        group['A'].iloc[1:] = new_nodes
        max_node = max_node+group.shape[0]-1
    return group

unique_AB_to_split = unique_AB_to_split.groupby('A_B').apply(insert_nodes)

In [106]:
unique_AB_to_split

,A,B,A_B,STOPA,STOPB,geometry
0,10002,936234,10002_10190,1,1,"LINESTRING (345534.000 5815270.000, 345404.800..."
1,936234,936235,10002_10190,1,1,"LINESTRING (345404.800 5815139.800, 345275.600..."
2,936235,936236,10002_10190,1,1,"LINESTRING (345275.600 5815009.600, 345146.400..."
3,936236,936237,10002_10190,1,1,"LINESTRING (345146.400 5814879.400, 345017.200..."
4,936237,10190,10002_10190,1,1,"LINESTRING (345017.200 5814749.200, 344888.000..."
...,...,...,...,...,...,...
56618,9995,975227,9995_24174,1,1,"LINESTRING (347516.000 5816663.000, 347710.000..."
56619,975227,24174,9995_24174,1,1,"LINESTRING (347710.000 5816637.500, 347904.000..."
56620,9996,975228,9996_24311,1,1,"LINESTRING (347846.000 5816180.000, 347706.333..."
56621,975228,975229,9996_24311,1,1,"LINESTRING (347706.333 5816111.667, 347566.667..."


In [108]:
unique_nodes = unique_AB_to_split.drop_duplicates(subset=['A'])[['A','geometry']].rename(columns={'A':'node'})
unique_nodes.geometry = unique_nodes.geometry.apply(lambda line: Point(line.coords[0]))
temp_B_nodes = unique_AB_to_split.drop_duplicates(subset=['B'])[['B','geometry']].rename(columns={'B':'node'})
temp_B_nodes.geometry = temp_B_nodes.geometry.apply(lambda line: Point(line.coords[-1]))
unique_nodes = unique_nodes.append(temp_B_nodes.loc[~temp_B_nodes['node'].isin(unique_nodes['node'].to_list())])
unique_nodes = gpd.GeoDataFrame(unique_nodes, geometry='geometry')

unique_nodes = unique_nodes.sort_values('node')

unique_nodes

,node,geometry
43903,7001,POINT (325194.000 5803164.000)
43906,7002,POINT (337874.000 5803155.000)
43908,7004,POINT (348811.000 5803118.000)
43913,7005,POINT (338246.000 5803111.000)
43919,7007,POINT (293931.000 5803099.000)
...,...,...
56615,975225,POINT (348982.750 5817697.500)
56617,975226,POINT (347390.000 5816680.500)
56619,975227,POINT (347710.000 5816637.500)
56621,975228,POINT (347706.333 5816111.667)


In [109]:
# Check for and drop duplicate geometries
# convert to wkb
unique_nodes["geometry"] = unique_nodes["geometry"].apply(lambda geom: geom.wkb)  
unique_nodes['reassign_node'] = unique_nodes['node']

group_list = []

for name, group in unique_nodes.groupby("geometry"):
    if group.shape[0] > 1:
        first_node = group.iloc[0]['node']
        group['reassign_node'] = first_node
    group_list.append(group)

unique_nodes = pd.concat(group_list, ignore_index=True)

# convert back to shapely geometry
unique_nodes["geometry"] = unique_nodes["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

/Users/frankiemacbook/opt/miniconda3/envs/py3/lib/python3.8/site-packages/geopandas/geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [110]:
unique_nodes

,node,geometry,reassign_node
0,8745,POINT (327680.000 5760458.000),8745
1,959670,POINT (344128.000 5806176.500),959670
2,959674,POINT (344128.000 5806176.500),959670
3,28428,POINT (327936.000 5812636.000),28428
4,7065,POINT (328320.000 5802745.000),7065
...,...,...,...
50899,966657,POINT (366270.500 5801312.000),966589
50900,941944,POINT (366271.000 5784827.333),941944
50901,969970,POINT (366271.000 5784827.333),941944
50902,962373,POINT (577022.000 5813080.667),962373


In [111]:
unique_AB_to_split = unique_AB_to_split.merge(unique_nodes[['node','reassign_node']], how='left', left_on='A', right_on='node', suffixes=('','_A'))
unique_AB_to_split = unique_AB_to_split.merge(unique_nodes[['node','reassign_node']], how='left', left_on='B', right_on='node', suffixes=('','_B'))

unique_AB_to_split['A'] = unique_AB_to_split['reassign_node']
unique_AB_to_split['B'] = unique_AB_to_split['reassign_node_B']
unique_AB_to_split = unique_AB_to_split.drop(['node','node_B','reassign_node','reassign_node_B'], axis=1)
# convert back to shapely geometry
# unique_nodes["geometry"] = unique_nodes["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

In [ ]:
# unique_nodes = unique_nodes.drop_duplicates(['reassign_node'])
# unique_nodes['node'] = unique_nodes['reassign_node']
# unique_nodes = unique_nodes.drop(['reassign_node'], axis=1)

# def closest_point(point):
#     distances = unique_nodes.distance(point).nsmallest(2)
#     distances = distances.loc[distances > 0]
#     nodes = unique_nodes.loc[distances.index,'node']
#     return [nodes.iloc[0], distances.iloc[0]]

# unique_nodes['nearest_node'] = unique_nodes['geometry'].apply(lambda row: pd.Series(closest_point(row)))

In [114]:
df_unique = df_unique.merge(unique_AB_to_split[['A_B','A','B','STOPA','STOPB','geometry']], how='left', on='A_B', suffixes=('','_merged')).reset_index(drop=True)

df_unique.loc[df_unique['A_merged'].notnull(),'A'] = df_unique.loc[df_unique['A_merged'].notnull(),'A_merged']
df_unique.loc[df_unique['A_merged'].notnull(),'B'] = df_unique.loc[df_unique['A_merged'].notnull(),'B_merged']
df_unique.loc[df_unique['A_merged'].notnull(),'STOPA'] = df_unique.loc[df_unique['A_merged'].notnull(),'STOPA_merged']
df_unique.loc[df_unique['A_merged'].notnull(),'STOPB'] = df_unique.loc[df_unique['A_merged'].notnull(),'STOPB_merged']
df_unique.loc[df_unique['A_merged'].notnull(),'geometry'] = df_unique.loc[df_unique['A_merged'].notnull(),'geometry_merged']
df_unique = df_unique.drop(['A_merged','B_merged','STOPA_merged','STOPB_merged','geometry_merged'], axis=1)

df_unique['A'] = df_unique['A'].apply(int)
df_unique['B'] = df_unique['B'].apply(int)

In [115]:
df_unique.loc[df_unique['A'] == df_unique['B']]

,A_B,A,B,STOPA,STOPB,LINKC_AM,BUS_SPD,DISTANCE,UNIQUE_ROUTE,UNIQUE_ROUTE_SEQ,LINKSEQ,mode,geometry


In [116]:
lasts = df_unique.reset_index().groupby('UNIQUE_ROUTE').last().reset_index().set_index('index')
lasts['last'] = True

df_unique = df_unique.merge(lasts['last'], how='left', left_index=True, right_index=True)

In [117]:
df_unique.loc[((df_unique['STOPB'] == 1) & (~df_unique['LINKC_AM'].isin([18,20,25])) & (df_unique['BUS_SPD'] != 80)) | df_unique['last'], 'stop_flag'] = True

In [ ]:
# distsum = 0
# total_rows = df_unique.shape[0]
# rows_processed = 0
# stopping_nodes = {}

# def find_nodes(row):
#     global distsum, stopping_nodes, rows_processed, start_node
#     a_node = row['A']
#     b_node = row['B']
#     route = row['UNIQUE_ROUTE']
#     rows_processed +=1
#     print('Finding nodes: %.1f' % (100 * rows_processed / total_rows) + '% processed', end='\r')
#     if distsum == 0:
#         stopping_nodes[a_node] = True
#         start_node = a_node
#         if rows_processed < total_rows:
#             if df_unique.loc[row.name+1,'UNIQUE_ROUTE'] == route:
#                 if start_node == df_unique.loc[row.name+1,'B']:
#                     stopping_nodes[b_node] = True
#                     return row
#     distsum += row['geometry'].length
#     if b_node in stopping_nodes:
#         distsum = 0
#         return row
#     if row['STOPB'] == 1:
#         if start_node == b_node:
#             return row
#         if rows_processed < total_rows:
#             if df_unique.loc[row.name+1,'UNIQUE_ROUTE'] == route:
#                 if distsum < 250:
#                     return row
#                 if (df_unique.loc[row.name+1,'B'] in stopping_nodes) and \
#                    (df_unique.loc[row.name+1,'geometry'].length < 200):
#                     return row
#         stopping_nodes[b_node] = True
#         distsum = 0
#     return row

# df_unique = df_unique.apply(lambda row: pd.Series(find_nodes(row)), axis=1)

# print('Finding nodes: finished           ')

In [118]:
distsum = 0
links = []
stops = []
total_rows = df_unique.shape[0]
rows_processed = 0

def consolidate_links_and_nodes(row):
    global distsum, links, stops, rows_processed
    a_node = row['A']
    b_node = row['B']
    stopa = 0
    stopb = 0
    totaldist = np.nan
    alllinks = np.nan
    allstops = np.nan
    rows_processed +=1
    print('Consolidating links and nodes: %.1f' % (100 * rows_processed / total_rows) + '% processed', end='\r')
    new_line = row['geometry']
    line_slice = new_line.coords
    if len(links) > 0:
        line_slice = new_line.coords[1:]
    links += [l for l in line_slice]
    if distsum == 0:
        stops.append((a_node,Point(links[0])))
        stopa = 1
    new_dist = new_line.length
    distsum += new_dist
    if row['stop_flag']:
        totaldist = distsum
        alllinks = LineString(links)
        stops.append((b_node,Point(links[-1])))
        stopb = 1
        allstops = stops
        distsum = 0
        links = []
        stops = []
    return [stopa, stopb, totaldist, alllinks, allstops, [Point(new_line.coords[0]),Point(new_line.coords[-1])], new_dist]

df_unique[['STOPA','STOPB','SEGDIST','SEGLINK','SEGSTOPS','LINKNODES','shape_dist_traveled']] = df_unique.apply(lambda row: pd.Series(consolidate_links_and_nodes(row)), axis=1)

print('Consolidating links and nodes: finished           ')

Consolidating links and nodes: finished           


In [119]:
def expand_points(group):
    first_line = group.iloc[0]
    first_line['LINKNODES'] = first_line['LINKNODES'][0]
    first_line['shape_dist_traveled'] = 0
    group['LINKNODES'] = group['LINKNODES'].apply(lambda x: x[1])
    group['shape_dist_traveled'] = (group['shape_dist_traveled'].cumsum()*100).apply(round)/100
    group['LINKSEQ'] = group['LINKSEQ'] + 1
    return first_line.to_frame().T.append(group)

df_shapes = df_unique[['UNIQUE_ROUTE','LINKSEQ','LINKNODES','shape_dist_traveled']]
df_shapes = df_shapes.groupby('UNIQUE_ROUTE').apply(expand_points).reset_index(drop=True)

df_shapes = gpd.GeoDataFrame(df_shapes, geometry='LINKNODES', crs='epsg:20255').to_crs('epsg:4326')

df_shapes['shape_pt_lon'],df_shapes['shape_pt_lat'] = zip(*df_shapes['LINKNODES'].apply(lambda x: [x.x,x.y]))

df_shapes = df_shapes.rename({'LINKDIST':'shape_dist_traveled','LINKSEQ':'shape_pt_sequence'}, axis=1).drop(['LINKNODES'], axis=1)

In [120]:
df = df.drop(['A','B','STOPA','STOPB'], axis=1).merge(df_unique[['UNIQUE_ROUTE_SEQ','A','B','STOPA','STOPB','SEGDIST','SEGLINK','SEGSTOPS']], how='left', on='UNIQUE_ROUTE_SEQ')

df = df.loc[(df.STOPA == 1) | (df.STOPB == 1)].sort_values(['year','LINENO','PERIOD','LINKSEQ'])

df.loc[df['STOPA'] == 0]

,PERIOD,mode,LINENO,LINKSEQ,NAME,LONGNAME,year,YEAR_LINE_PERIOD,YEAR_LINE,A_B,...,geometry,UNIQUE_ROUTE,UNIQUE_ROUTE_SEQ,A,B,STOPA,STOPB,SEGDIST,SEGLINK,SEGSTOPS


In [121]:
df['STOPA_shift'] = df['STOPA'].shift()
df['A_shift'] = df['A'].shift()

firsts = df.groupby('NAME').first()
firsts.loc[firsts['STOPA'] == 0]

,PERIOD,mode,LINENO,LINKSEQ,LONGNAME,year,YEAR_LINE_PERIOD,YEAR_LINE,A_B,LINKC_AM,...,UNIQUE_ROUTE_SEQ,A,B,STOPA,STOPB,SEGDIST,SEGLINK,SEGSTOPS,STOPA_shift,A_shift
NAME,,,,,,,,,,,,,,,,,,,,,


In [122]:
df['A'] = np.where(df['STOPA'] == 0, df['A_shift'], df['A'])
df['STOPA'] = np.where(df['STOPA'] == 0, df['STOPA_shift'], df['STOPA'])

df = df.loc[df.STOPB == 1].sort_values(['year','LINENO','PERIOD','LINKSEQ'])
df['A'] = df['A'].apply(int)
df['B'] = df['B'].apply(int)

df = df.drop(['LINKSEQ','UNIQUE_ROUTE_SEQ'],axis=1)

df.loc[df['A'] == df['B']]

,PERIOD,mode,LINENO,NAME,LONGNAME,year,YEAR_LINE_PERIOD,YEAR_LINE,A_B,LINKC_AM,...,UNIQUE_ROUTE,A,B,STOPA,STOPB,SEGDIST,SEGLINK,SEGSTOPS,STOPA_shift,A_shift


In [123]:
pt_reporting['Time Period'] = pt_reporting['Time Period'].apply(lambda x: x.split(' ')[0])
pt_reporting['YEAR_NAME_PERIOD'] = pt_reporting['year'] + '_' + pt_reporting['Short Name'] + "_" + pt_reporting['Time Period']

In [124]:
df['YEAR_NAME_PERIOD'] = df['year'] + '_' + df['NAME'] + "_" + df['PERIOD']
unique_speeds_headways = df.groupby('YEAR_NAME_PERIOD').first().reset_index()
unique_speeds_headways = unique_speeds_headways.merge(pt_reporting[['YEAR_NAME_PERIOD','Headway','Average Speed']], how='left', on='YEAR_NAME_PERIOD').drop('YEAR_NAME_PERIOD', axis=1)[['YEAR_LINE','year','LINENO','PERIOD','NAME','LONGNAME','mode','Headway','Average Speed','UNIQUE_ROUTE']]
unique_speeds_headways = unique_speeds_headways.rename({'LINENO':'id','NAME':'route','LONGNAME':'name','PERIOD':'period','Average Speed':'speed','Headway':'headway'}, axis=1)

In [125]:
unique_speeds_headways

,YEAR_LINE,year,id,period,route,name,mode,headway,speed,UNIQUE_ROUTE
0,2018_305,2018,305,AM,109,BOX HILL - PORT MELBOURNE,tram,7.5,15.42,1457
1,2018_305,2018,305,IP,109,BOX HILL - PORT MELBOURNE,tram,10.0,16.24,1457
2,2018_305,2018,305,OP,109,BOX HILL - PORT MELBOURNE,tram,12.0,16.54,1457
3,2018_305,2018,305,PM,109,BOX HILL - PORT MELBOURNE,tram,7.5,15.98,1457
4,2018_2,2018,2,AM,11011,Mernda - Flinders Street - All Stations,train,13.3,36.16,785
...,...,...,...,...,...,...,...,...,...,...
31625,2051_1476,2051,1476,PM,WredstoneR,Sunbury - Redstone Hill,bus,40.0,20.09,426
31626,2051_1475,2051,1475,AM,Wredstone,Sunbury - Redstone Hill,bus,40.0,21.36,1307
31627,2051_1475,2051,1475,IP,Wredstone,Sunbury - Redstone Hill,bus,40.0,21.54,1307
31628,2051_1475,2051,1475,OP,Wredstone,Sunbury - Redstone Hill,bus,60.0,21.72,1307


In [126]:
pivot = pd.pivot_table(unique_speeds_headways, index=['YEAR_LINE'], columns=["period"], values=["headway","speed"], aggfunc=np.mean)
pivot.columns = ['_'.join(col).strip().lower() for col in pivot.columns.values]

In [127]:
speed_cols = ['speed_am','speed_ip','speed_pm','speed_op']
headway_cols = ['headway_am','headway_ip','headway_pm','headway_op']
unique_speeds_headways = unique_speeds_headways.merge(pivot.reset_index(), how='left', on='YEAR_LINE').groupby('YEAR_LINE').first().reset_index().replace(np.nan,0).drop('period',axis=1)

In [128]:
def fix_missing_speeds(row):
    for i in range(len(speed_cols)):
        if row[speed_cols[i]] == 0:
            row[speed_cols[i]] = row[speed_cols[(i+3) % 4]]
    return row

unique_speeds_headways = unique_speeds_headways.apply(fix_missing_speeds, axis=1)
unique_speeds_headways = unique_speeds_headways.loc[(unique_speeds_headways[headway_cols] != 0).any(1)]

In [129]:
def stop_nodes(group):
    global stop_list, link_list, dist_list, node_list, counter
    counter = 0
    link_list = []
    stop_list = []
    dist_list = []
    node_list = []
    group.apply(get_nodes, axis=1)
    return [list(zip(link_list, node_list, dist_list)), MultiLineString(link_list), stop_list]
    
def get_nodes(row):
    global stop_list, link_list, dist_list, node_list, counter
    link_list.append(row['SEGLINK'])
    node_list.append(str(row['A'])+'_'+str(row['B']))
    dist_list.append(row['SEGDIST'])
    if counter == 0:
        stop_list.append(row['SEGSTOPS'][0])
    stop_list.append(row['SEGSTOPS'][1])
    counter += 1
    
def number_segments(l):
    segs = ''
    for i in range(len(l)):
        segs += str(i)+','
    return segs[:-1]

stop_patterns = df.loc[df['YEAR_LINE_PERIOD'].isin(unique_routes['YEAR_LINE_PERIOD'].to_list())].groupby('UNIQUE_ROUTE').apply(stop_nodes).reset_index()
stop_patterns[['SEGMENTS','LINE','STOPPATTERN']] = pd.DataFrame(stop_patterns[0].tolist(), index=stop_patterns.index)
stop_patterns = stop_patterns.drop(0, axis=1)
stop_patterns['segments'] = stop_patterns['SEGMENTS'].apply(number_segments)

In [130]:
stop_patterns = unique_speeds_headways.merge(stop_patterns, how='left', on='UNIQUE_ROUTE')

# Replace non-ASCII characters
table = {0x2013: '-', 0x2014: '--', 0x00a7: 'sect. ', 0x00A0: ' '}
stop_patterns.loc[~stop_patterns['name'].apply(lambda string: string.isascii()),'name'] = stop_patterns.loc[~stop_patterns['name'].apply(lambda string: string.isascii()),'name'].apply(lambda string: string.translate(table))
stop_patterns.loc[~stop_patterns['name'].apply(lambda string: string.isascii()),'name']

Series([], Name: name, dtype: object)

In [131]:
route_output = stop_patterns.groupby('YEAR_LINE').first().reset_index()[['year','id','route','mode','name','LINE']]
route_output = route_output.rename({'LINE':'geometry','mode':'OPERATOR_N','route':'ROUTE_SHORT','name':'ROUTE_LONG','id':'ROUTE_ID'}, axis=1)
route_output['SHAPE_ID'] = route_output['ROUTE_ID']

route_output = gpd.GeoDataFrame(route_output[['year','SHAPE_ID','ROUTE_ID','ROUTE_SHORT','ROUTE_LONG','OPERATOR_N','geometry']], geometry='geometry', crs='epsg:20255').to_crs('epsg:4326')

In [132]:
routes = stop_patterns[['year','route','id','mode','name']+headway_cols+speed_cols+['segments']]

In [133]:
seg_expand = stop_patterns[['id','year','mode','route','name','UNIQUE_ROUTE']] \
    .merge(stop_patterns.SEGMENTS.explode(), right_index = True, left_index = True) \
    .dropna() \
    .sort_values(['year','id']) \
    .reset_index(drop = True)

In [134]:
seg_expand[['SEGMENT','NODES','route_dist']] = pd.DataFrame(seg_expand['SEGMENTS'].tolist(), index=seg_expand.index)
seg_expand = gpd.GeoDataFrame(seg_expand.drop('SEGMENTS', axis=1).rename(columns={'SEGMENT':'geometry'}), geometry='geometry', crs='epsg:20255').to_crs('epsg:4326')

In [135]:
seg_expand['NODES'] = seg_expand['NODES'].apply(lambda x: x.split('_'))
seg_expand[['stop1','stop2']] = pd.DataFrame(seg_expand['NODES'].tolist(), index=seg_expand.index)
seg_expand = seg_expand.drop('NODES', axis=1)
seg_expand['stop1N'] = seg_expand['stop1'].apply(str)
seg_expand['stop2N'] = seg_expand['stop2'].apply(str)
seg_expand = seg_expand.rename({'id':'routes'}, axis=1)
seg_expand['id'] = seg_expand.groupby(['year','routes']).cumcount()

In [136]:
(seg_expand['stop1'] == seg_expand['stop2']).to_frame().groupby(0)[0].count()

0
False    962461
Name: 0, dtype: int64

In [137]:
sp_expand = stop_patterns[['id','year','mode','route','name']] \
    .merge(stop_patterns.STOPPATTERN.explode(), right_index = True, left_index = True) \
    .dropna() \
    .sort_values(['year','id']) \
    .reset_index(drop = True)

In [138]:
sp_expand[['id','STOPPATTERN']] = pd.DataFrame(sp_expand['STOPPATTERN'].tolist(), index=sp_expand.index)
sp_expand['name'] = sp_expand['id'].apply(str)
sp_expand = gpd.GeoDataFrame(sp_expand.rename(columns={'STOPPATTERN':'geometry'}), geometry='geometry', crs='epsg:20255').to_crs('epsg:4326')
sp_expand = sp_expand.drop_duplicates(subset=['id']).drop(['year','route','mode'], axis=1).reset_index(drop=True)

In [139]:
end_processing_time = time.time()
pre_processing_time = end_processing_time - start_time

# from importlib import reload
# reload(create_gtfs_from_basicinfo)

# print('Saving network shapefiles')
# if not os.path.isdir('output/Network'):
#     os.mkdir('output/Network')
# route_output.to_file('output/Network/VITM_'+year+'_segments.shp')
# sp_expand.to_file('output/Network/VITM_'+year+'_stops.shp')

if not os.path.isdir('input/temp'):
    os.mkdir('input/temp')

modes = {'vline':'1','train':'2','tram':'3','bus':'4','skybus':'11'}

for year in years:
    print('Generating GTFS outputs for '+year+' reference case\n')
    for mode in all_modes:
        if mode not in modes:
            print("'"+mode+"' not a valid mode")
            continue
#         if mode not in seg_expand['mode'].unique().to_list():
#             continue
    #     if mode != 'train':
    #         continue
        sys.argv = ['', '--year', year, '--service', mode]

        if not os.path.isdir('output/'+modes[mode]):
            os.mkdir('output/'+modes[mode])

        stop_ids = seg_expand.loc[(seg_expand['mode'] == mode) & (seg_expand['year'] == year), ['stop1','stop2']].values
        stop_ids = list(dict.fromkeys([int(val) for sublist in stop_ids for val in sublist]))
        
        unique_route_ids = list(seg_expand.loc[(seg_expand['mode'] == mode) & (seg_expand['year'] == year), 'UNIQUE_ROUTE'].unique())

        sp_expand.loc[sp_expand['id'].isin(stop_ids)].to_file('input/temp/VITM_stops.shp')
        seg_expand.loc[(seg_expand['mode'] == mode) & (seg_expand['year'] == year)].drop('UNIQUE_ROUTE',axis=1).to_file('input/temp/VITM_segments.shp')
        routes.loc[(routes['mode'] == mode) & (routes['year'] == year)].to_csv('input/temp/VITM_routes.csv', index=False, sep=';')
        df_shapes.loc[df_shapes['UNIQUE_ROUTE'].isin(unique_route_ids)].drop('UNIQUE_ROUTE',axis=1).to_csv('input/temp/shapes.txt', index=False)

        create_gtfs_from_basicinfo.run()

    print('\nCreating output/VITM_'+year+'_GTFS.zip...\n')
    # create a ZipFile object
    with zipfile.ZipFile('output/VITM_'+year+'_GTFS.zip', 'w', zipfile.ZIP_DEFLATED) as zipObj:
        # Iterate over all the files in directory
        for folder_id in list(modes.values()):
            if os.path.isdir('output/'+folder_id):
                for folderName, subfolders, filenames in os.walk('output/'+folder_id):
                    for filename in filenames:
                        #create complete filepath of file in directory
                        filePath = os.path.join(folderName, filename)
                        # Add file to zip
                        zipObj.write(filePath, os.path.join(folder_id, filename))
                shutil.rmtree('output/'+folder_id)
    
print('Cleaning up temp files...')
shutil.rmtree('input/temp')

print('Done')

total_time = time.time() - start_time

print('Pre-processing time: %.1f minutes' % (pre_processing_time / 60))
print('Total time: %.1f minutes' % (total_time / 60))

os.system("printf '\a'")

Generating GTFS outputs for 2018 reference case

Creating GTFS feed for bus
Processing routes 0 to 907
About to save complete timetable to file output/4/google_transit.zip ...
...finished writing to file output/4/google_transit.zip.
Creating GTFS feed for skybus
Processing routes 0 to 17
About to save complete timetable to file output/11/google_transit.zip ...
...finished writing to file output/11/google_transit.zip.
Creating GTFS feed for train
Processing routes 0 to 267
About to save complete timetable to file output/2/google_transit.zip ...
...finished writing to file output/2/google_transit.zip.
Creating GTFS feed for tram
Processing routes 0 to 45
About to save complete timetable to file output/3/google_transit.zip ...
...finished writing to file output/3/google_transit.zip.
Creating GTFS feed for vline
Processing routes 0 to 95
About to save complete timetable to file output/1/google_transit.zip ...
...finished writing to file output/1/google_transit.zip.

Creating output/VITM_20

KeyboardInterrupt: 